In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def moving_avg(arr, window=3):
    arr2 = np.zeros_like(arr).astype(np.float32)
    for i in range(0,window-1):
        arr2[i] = arr[:i+1].mean()
    for i in range(window-1,len(arr)):
        arr2[i] = arr[i-window+1:i+1].mean()
    return arr2
#moving_avg(input_arr[1],5)
#input_arr
    

In [ ]:
mycore = 4
desiredslice=3
datalist = pickle.load(open('core'+str(mycore)+'ToSlice'+str(desiredslice)+'.pkl', 'rb'))

In [ ]:
targets = []
inputs = []
offset = 31
threshold = 40+offset
for data in datalist:
    for i in range(1,len(data)):
        bit, trace = data[i]
        if data[i-1][0] == 0:
            continue
        if len(trace) >= threshold:
            targets.append(bit)
            inputs.append(list(trace[:threshold]))
        elif len(trace) >= threshold//2 and i < len(data)-1 and len(data[i+1][1]) + len(trace) >= threshold:
            pass
            #targets.append(bit)
            #inputs.append(list(trace) + list(data[i+1][1][:threshold-len(trace)]))
            
target_arr = np.array(targets)
input_arr = np.array(inputs)
print(len(input_arr))
print(target_arr.sum())

In [ ]:

fig = plt.figure(figsize=(15,15))
seed = np.random.randint(len(targets)-10)
for idx in range(1,10):
    rn = seed + idx
    ax1 = fig.add_subplot(3,3,idx)
    ax1.set_title(str(targets[rn]))
    xaxis = np.arange(1,threshold+1-31)
    yaxis = moving_avg(input_arr[rn][31:],3)
    #yaxis = input_arr[rn]
    print(input_arr[rn].mean())
    ax1.plot(xaxis,yaxis)
    axes = plt.gca()
    #axes.set_ylim([145,220])
fig.show()

In [ ]:
train_x = []
train_y = []
test_x = []
test_y = []

mn = input_arr.mean()
st = input_arr.std()

for i in range(len(target_arr)):
    #mn = input_arr[i].mean()
    #st = input_arr[i].std()
    xdata = (input_arr[i]-mn)/st
    if i%4 == 0:
        test_x.append(xdata[31:])
        test_y.append(target_arr[i])
    else:
        train_x.append(xdata[31:])
        train_y.append(target_arr[i])
len(test_x)

In [ ]:
from sklearn import svm
clf = svm.SVC()
clf.fit(train_x, train_y)
pred_y = clf.predict(test_x)
#tpred_y = clf.predict(train_x)
(pred_y == test_y).sum()/len(pred_y)
#(tpred_y == train_y).sum()/len(train_y)

In [ ]:
from sklearn.metrics import f1_score
f1_score(test_y, pred_y)

In [ ]:
lens = [(len(d[1]),d[0]) for d in data]

In [ ]:
lens

In [ ]:
input_arr.mean()

In [ ]:
import torch
def shifter(arr, window=32):
    dup = arr.expand(arr.size(0), arr.size(1)+1, arr.size(1))
    dup2 = dup.reshape(arr.size(0), arr.size(1), arr.size(1)+1)
    shifted = dup2[:,:,:window]
    return shifted
print(shifter(torch.arange(42).view(1,42)))

In [ ]:
gaup = [2.46, 3.08, 3.80, 4.62, 5.43, 6.21, 6.95, 7.75]
gaup = np.array(gaup)*8
gaua = [0.612, 0.598, 0.581, 0.573, 0.565, 0.571, 0.553, 0.554]

sinp = [2.03, 2.94, 3.87, 4.67, 5.70, 6.69, 7.51, 8.48]
sinp = np.array(sinp)*8
sina = [0.659, 0.614, 0.603, 0.592, 0.576, 0.568, 0.566, 0.556]

advp = [2.51, 2.58, 2.71, 4.15, 5.22, 6.32, 7.30, 7.41]
advp = np.array(advp)*8
adva = [0.528, 0.511, 0.507, 0.501, 0.5, 0.5, 0.5, 0.5]


plt.plot(gaup, gaua, label = "Gaussian")

plt.plot(sinp, sina, label = "Gaussian Sinusoid")

plt.plot(advp, adva, label = "Adversarial ML")

plt.xlabel('Avg. added cycles')
# Set the y axis label of the current axis.
plt.ylabel('Classifier Accuracy')
# Set a title of the current axes.
plt.title('Mask generator comparison')
# show a legend on the plot
plt.legend()
# Display a figure.
plt.show()